In [13]:
# Import the quantum chemistry driver and related units
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Import the qubit mapper library for Ci+ and Ci
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper, BravyiKitaevSuperFastMapper

# Import a classical analyzer to compare results with VQE
from qiskit_algorithms import NumPyMinimumEigensolver

# Import VQE and related libraries
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, NELDER_MEAD, COBYLA
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# Import ground state solver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [15]:
mapper = JordanWignerMapper()
# mapper = BravyiKitaevMapper()

In [26]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, NELDER_MEAD())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [27]:
ansatz.decompose().draw()

┌───┐┌───────────────────────────────┐┌───────────────────────────────┐»
q_0: ┤ X ├┤0                              ├┤0                              ├»
     ├───┤│                               ││                               │»
q_1: ┤ X ├┤1                              ├┤1                              ├»
     ├───┤│  exp(-it (IIIY + IIZY))(t[0]) ││  exp(-it (IYII + ZYZI))(t[1]) │»
q_2: ┤ X ├┤2                              ├┤2                              ├»
     └───┘│                               ││                               │»
q_3: ─────┤3                              ├┤3                              ├»
          └───────────────────────────────┘└───────────────────────────────┘»
«     ┌─────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                        ├
«     │                                                                         │
«q_1: ┤1                                                                        ├
«     │  exp(-it (ZXZY + IYZX + IXIY + ZYIX + IYIX + ZXIY + ZYZX + IXZY))(t[2]) │
«q_2: ┤2                                                                        ├
«     │                                                                         │
«q_3: ┤3                                                                        ├
«     └─────────────────────────────────────────────────────────────────────────┘

In [28]:
calc = GroundStateEigensolver(mapper, vqe_solver)

In [29]:
%%time
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275028293
  - computed part:      -1.857275028293
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306033844
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.389017800444]
    - computed part:      [0.0  0.0  1.389017800444]
  > Dipole moment (a.u.): [0.0  0.0  -0.000069100444]  Total: 0.000069100444
                 (debye): [0.0  0.0  -0.000175635794]  Total: 0.000175635794
 
CPU times: user 12.4 s, sys: 307 ms, total: 12.7 s
Wall time: 12.7 s


### Comparing with a classical Eigensolver (provided by numpy)

In [74]:
numpy_solver = NumPyMinimumEigensolver()
calc2 = GroundStateEigensolver(mapper, numpy_solver)

In [75]:
%%time
res2 = calc2.solve(es_problem)
print(res2)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 
CPU times: user 286 ms, sys: 20.3 ms, total: 306 ms
Wall time: 303 ms


### Measuring performance of UCCSD with various interactomic distance